# Pre-process all the data

In [1]:
# Corpus reader:
import numpy as np
import random

import os
root = './Confs_newline/Conf1/'
from nltk.corpus.reader import CategorizedPlaintextCorpusReader
reader = CategorizedPlaintextCorpusReader(root, r'.*\.txt', cat_pattern=r'(\w+)/*', encoding='utf-8')

In [2]:
print(reader.categories())
print(reader.fileids())

['kiz', 'kork', 'mutlu']
['kiz.txt', 'kork.txt', 'mutlu.txt']


In [3]:
### First, tokenize Punctuation: 
# create a token dictionary:
punc_dict= {'.':'||PERIOD||', ',': '||COMMA||', '"': '||QUOTATION_MARK||', ';': '||SEMICOLON||',
                '!': '||EXCLAMATION_MARK||', '?': '||QUESTION_MARK||', '(': '||LEFT_PAREN||',
                ')': '||RIGHT_PAREN||', '?': '||QUESTION_MARK||', 
                '\n': '||NEW_LINE||', '-': '||DASH||'}

In [4]:
def sent_tokenize_whole_tweets(text): # raw text --> whole tweets file content
    for key, token in punc_dict.items():
        text = text.replace(key, ' {} '.format(token))

    sentences= []
    for line in text.split('||NEW_LINE||'):
        line= line.strip()
        sentences.append(line)
    return sentences

In [5]:
all_text=[]
labels= []

In [6]:
for label,file_name in zip(reader.categories(), reader.fileids()):
    sentences= sent_tokenize_whole_tweets(reader.raw(file_name)) # --> this should return a list of contents
    labels.extend([label for i in sentences])
    all_text.extend([i.lower() for i in sentences])

# Now, we have all tweets in all_text list!

## Transforming Text into Numbers

In [7]:
word_counts={}
for i in range(len(all_text)):
    for word in all_text[i].split(" "):
        word_counts[word] = word_counts.get(word,0) +1

vocab = set(word_counts.keys())
vocab_size = len(vocab)

print("Number of unique words: {} ".format(vocab_size))
print("Number of tweets: {}".format(len(all_text)))
assert len(all_text)== len(labels), "Each tweet should have a label."

sorted_word_counts= sorted(word_counts, key= word_counts.get, reverse= True)

int_to_vocab= {ii: word for ii,word in enumerate(sorted_word_counts)}
vocab_to_int= {word: ii for ii, word in int_to_vocab.items()}

Number of unique words: 7426 
Number of tweets: 2170


In [8]:
all_text[2]

'hayır çok severim diziyi yani aşırı kızdım'

In [9]:
int_to_vocab[0]

''

# Reducing Noise in the Input Data

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

$$ P(0) = 1 - \sqrt{\frac{1*10^{-5}}{1*10^6/16*10^6}} = 0.98735 $$

In [10]:
int_words = [vocab_to_int[word] for word in vocab]
print(int_words[:5])

[0, 2843, 3036, 3635, 5873]


In [11]:
threshold = 1e-4

word_counts_intwords = {vocab_to_int[word]:count for word,count in word_counts.items()}
total_count = vocab_size
freqs = {word: count/total_count for word, count in word_counts_intwords.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts_intwords}
# discard some frequent words, according to the subsampling equation
# create a new list of words for training

selected_words = [word for word in word_counts_intwords if random.random() < (1 - p_drop[word])]
print(selected_words[:30])

[2139, 71, 2140, 291, 2142, 2143, 2145, 2146, 842, 631, 843, 1226, 172, 2147, 2148, 2149, 2150, 844, 1227, 9, 1228, 2151, 2153, 636, 1230, 637, 251, 2155, 1231, 2156]


In [12]:
len(selected_words)

5619

In [13]:
all_text[0]

'çok kızdım ne kadar ayıp şey'

In [14]:
for i in range(len(all_text)):
    line_split= all_text[i].split(" ")
    line_split_reduced= [word for word in line_split if vocab_to_int[word] in selected_words]
    all_text[i]= ' '.join(line_split_reduced)


# Delete empty tweets:

In [15]:
len(all_text)

2170

In [16]:
all_text= [line for line in all_text if len(line)>0]
labels= [labels[i] for i in range(len(all_text)) if len(all_text[i])>0]

print("Number of tweets after reducing some noise: {}".format(len(all_text)))
assert len(all_text)== len(labels), "Each tweet should have a label."

Number of tweets after reducing some noise: 2077


# Encoding the words
The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

In [17]:
word_counts={}
for i in range(len(all_text)):
    for word in all_text[i].split(" "):
        word_counts[word] = word_counts.get(word,0) +1

vocab = set(word_counts.keys())
vocab_size = len(vocab)
print("Number of unique words: {} ".format(vocab_size))

sorted_word_counts= sorted(word_counts, key= word_counts.get, reverse= True)

int_to_vocab= {ii: word for ii,word in enumerate(sorted_word_counts, 0)} #start from 1.
vocab_to_int= {word: ii for ii, word in int_to_vocab.items()}


Number of unique words: 5619 


In [18]:
## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
encoded_tweets = []
for tweet in all_text:
    encoded_tweets.append([vocab_to_int[word] for word in tweet.split(" ")])

# Encoding the labels

In [19]:
def map_label(l):
    if l== 'kiz':
        return 0
    elif l== 'kork':
        return 1
    return 2  

In [20]:
encoded_labels = np.array([map_label(label) for label in labels])

# Word2Vec model:

In [21]:
import numpy as np
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = words[start:idx] + words[idx+1:stop+1]
    
    return list(target_words)

In [22]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [23]:
x,y = next(get_batches(encoded_tweets, batch_size=4, window_size=5))

print('x\n', x)
print('y\n', y)

x
 [[1022, 5], [1022, 5], [1023, 49, 1024, 1025, 1026, 1027, 256], [1023, 49, 1024, 1025, 1026, 1027, 256], [1023, 49, 1024, 1025, 1026, 1027, 256], [159, 257], [159, 257], [435, 17, 1028, 1029, 1030, 1031, 258, 436], [435, 17, 1028, 1029, 1030, 1031, 258, 436], [435, 17, 1028, 1029, 1030, 1031, 258, 436]]
y
 [[1023, 49, 1024, 1025, 1026, 1027, 256], [159, 257], [1022, 5], [159, 257], [435, 17, 1028, 1029, 1030, 1031, 258, 436], [1023, 49, 1024, 1025, 1026, 1027, 256], [435, 17, 1028, 1029, 1030, 1031, 258, 436], [1022, 5], [1023, 49, 1024, 1025, 1026, 1027, 256], [159, 257]]


In [24]:
extended_tweets= []
for line in encoded_tweets:
    extended_tweets.extend(line)

In [25]:
extended_tweets

[1022,
 5,
 1023,
 49,
 1024,
 1025,
 1026,
 1027,
 256,
 159,
 257,
 435,
 17,
 1028,
 1029,
 1030,
 1031,
 258,
 436,
 1,
 437,
 437,
 1032,
 1033,
 160,
 438,
 161,
 38,
 1034,
 439,
 1035,
 1036,
 1037,
 1038,
 1039,
 1040,
 1041,
 162,
 1042,
 257,
 1043,
 0,
 440,
 1044,
 1045,
 99,
 1046,
 1047,
 100,
 1048,
 0,
 17,
 126,
 1049,
 1050,
 56,
 1051,
 1052,
 1053,
 1054,
 441,
 84,
 442,
 442,
 443,
 1055,
 18,
 127,
 1056,
 1057,
 1058,
 259,
 1059,
 101,
 0,
 1060,
 163,
 1061,
 1062,
 444,
 1063,
 1064,
 64,
 445,
 446,
 1065,
 1066,
 446,
 445,
 1067,
 447,
 65,
 0,
 448,
 448,
 1068,
 39,
 102,
 0,
 1069,
 0,
 1070,
 260,
 0,
 1071,
 449,
 1072,
 1073,
 5,
 450,
 128,
 1074,
 1075,
 19,
 1076,
 261,
 20,
 1077,
 25,
 1078,
 85,
 1079,
 1080,
 262,
 1081,
 1082,
 1083,
 1084,
 1085,
 1086,
 1087,
 1088,
 164,
 129,
 40,
 103,
 165,
 1089,
 1090,
 263,
 1091,
 451,
 166,
 264,
 1092,
 1093,
 265,
 1094,
 266,
 267,
 452,
 1095,
 32,
 1096,
 1097,
 453,
 454,
 167,
 1098,
 1099,

# The model:
- The input words are passed in as batches of input word tokens.
- This will go into a hidden layer of linear units (our embedding layer).
- Then, finally into a softmax output layer.

## SkipGram model

An Embedding layer takes in a number of inputs, importantly:
* **num_embeddings** – the size of the dictionary of embeddings, or how many rows you'll want in the embedding weight matrix
* **embedding_dim** – the size of each embedding vector; the embedding dimension

In [26]:
def cosine_similarity(embedding, valid_size=16, valid_window=10, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the embedding matrix.
        Here, embedding should be a PyTorch embedding module.
    """
    
    # Here we're calculating the cosine similarity between some random words and 
    # our embedding vectors. With the similarities, we can look at what words are
    # close to our random words.
    
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # pick N words from our ranges (0,window) and (1000,1000+window). lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

In [27]:
import torch
from torch import nn
import torch.optim as optim

In [28]:
class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        
        self.embed = nn.Embedding(n_vocab, n_embed)
        self.output = nn.Linear(n_embed, n_vocab)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.embed(x)
        scores = self.output(x)
        log_ps = self.log_softmax(scores)
        
        return log_ps

### Training
Because we applied a softmax function to our model output, we are using NLLLoss** as opposed to cross entropy. This is because Softmax  in combination with NLLLoss = CrossEntropy loss .

In [29]:
# check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_dim= 200 # you can change, if you want

model = SkipGram(len(vocab_to_int), embedding_dim).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

print_every = 30
steps = 10
epochs = 1000

# train for some number of epochs
for e in range(epochs):
    
    # get input and target batches
    for inputs, targets in get_batches(extended_tweets, 32):
        steps += 1
        inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
        inputs, targets = inputs.to(device), targets.to(device)
        
        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if steps % print_every == 0:                  
            # getting examples and similarities      
            valid_examples, valid_similarities = cosine_similarity(model.embed, device=device)
            _, closest_idxs = valid_similarities.topk(3) # topk highest similarities
            
            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...")

gitti | taban, sürdü
şey | konu, izleyenler
sıçradı | sıkkın, çıkamıyorum
aklım | yattım, gidiyorum
yerinde | şerefsizsen, gözlerinizi
bozuldu | oflattım, abone
ay | susma, çıkardığım
alnından | oysaki, eğri
anlamlı | alarak, sürünerek
namjoon | polise, sersem
isterim | veli, saçma
harikalar | karanlıkla, etraftan
perşembe | denedik, boğazına
sesler | lütfen, kapıya
dileriz | koşulsuz, yosun
olayım | ağlamalarını, camdan
...
ve | karne, görüntüler
alnından | oysaki, eğri
aklım | yattım, gidiyorum
ay | susma, çıkardığım
yerinde | şerefsizsen, gözlerinizi
gitti | taban, sürdü
bozuldu | oflattım, abone
bir | konan, turuncu
tadından | moda, seviyorummm
olayım | ağlamalarını, camdan
sesler | lütfen, kapıya
perşembe | denedik, boğazına
 | soğukluğu, okula
isterim | veli, saçma
dileriz | koşulsuz, yosun
namjoon | polise, sersem
...
ay | susma, çıkardığım
sıçradı | sıkkın, çıkamıyorum
alnından | oysaki, eğri
aklım | yattım, gidiyorum
ve | karne, görüntüler
gitti | taban, sürdü
şey | konu, izl

yerinde | meme, gözlerinizi
ay | bırakmam, tanıdığım
aklım | gidiyorum, yattım
ve | karne, görüntüler
sıçradı | sıkkın, çıkamıyorum
alnından | oysaki, pasta
şey | konu, izleyenler
gitti | taban, sürdü
namjoon | polise, korkuluklundan
anlamlı | baktığında, alarak
sesler | lütfen, itin
harikalar | etraftan, karanlıkla
perşembe | denedik, getirmiyo
tadından | seviyorummm, gri
dileriz | koşulsuz, yosun
isterim | veli, shfjkf
...
şey | konu, izleyenler
bir | konan, turuncu
yerinde | meme, gözlerinizi
alnından | oysaki, eğri
ay | bırakmam, tanıdığım
aklım | gidiyorum, yattım
sıçradı | sıkkın, çıkamıyorum
gitti | taban, sürdü
dileriz | koşulsuz, yosun
sesler | lütfen, itin
harikalar | etraftan, karanlıkla
tadından | seviyorummm, gri
isterim | veli, shfjkf
perşembe | denedik, getirmiyo
anlamlı | baktığında, alarak
 | soğukluğu, okula
...
bir | konan, turuncu
gitti | sürdü, taban
yerinde | meme, gözlerinizi
ve | karne, görüntüler
aklım | gidiyorum, yattım
bozuldu | abone, sizde
alnından | oysa

şey | konu, izleyenler
bozuldu | abone, sizde
sıçradı | sıkkın, çıkamıyorum
aklım | gidiyorum, yattım
ve | karne, görüntüler
ay | tanıdığım, bırakmam
yerinde | meme, gözlerinizi
bir | konan, turuncu
namjoon | polise, korkuluklundan
 | soğukluğu, okula
sesler | itin, lütfen
harikalar | etraftan, grammy
tadından | seviyorummm, shshshsh
olayım | camdan, ağlamalarını
anlamlı | sürünerek, baktığında
perşembe | getirmiyo, denedik
...
sıçradı | sıkkın, çıkamıyorum
alnından | oysaki, pasta
bozuldu | abone, sizde
ve | karne, görüntüler
aklım | gidiyorum, yattım
bir | konan, turuncu
ay | tanıdığım, bırakmam
şey | konu, izleyenler
perşembe | getirmiyo, denedik
dileriz | koşulsuz, elettirik
olayım | camdan, ağlamalarını
harikalar | etraftan, kitap
isterim | veli, shfjkf
tadından | seviyorummm, shshshsh
anlamlı | sürünerek, baktığında
 | soğukluğu, okula
...
alnından | oysaki, pasta
bozuldu | abone, sizde
bir | konan, turuncu
gitti | sürdü, majahahajajajajajajjjajajanznznznzjznz
ay | tanıdığım, b

yerinde | meme, gözlerinizi
gitti | sürdü, majahahajajajajajajjjajajanznznznzjznz
aklım | gidiyorum, yattım
sıçradı | sıkkın, çıkamıyorum
alnından | pasta, kardeşsimin
bir | konan, farkındayım
ve | karne, görüntüler
ay | tanıdığım, bırakmam
anlamlı | baktığında, sürünerek
tadından | seviyorummm, shshshsh
perşembe | getirmiyo, ayak
 | soğukluğu, okula
harikalar | etraftan, kitap
isterim | veli, taşındık
dileriz | elettirik, koşulsuz
olayım | camdan, aramda
...
sıçradı | sıkkın, çıkamıyorum
alnından | kardeşsimin, pasta
bozuldu | salıca, sizde
ve | karne, görüntüler
gitti | sürdü, majahahajajajajajajjjajajanznznznzjznz
şey | konu, pürüzsüz
bir | konan, farkındayım
aklım | gidiyorum, yattım
perşembe | getirmiyo, ayak
sesler | itin, has
tadından | seviyorummm, shshshsh
dileriz | elettirik, koşulsuz
 | soğukluğu, okula
anlamlı | baktığında, sürünerek
namjoon | polise, yönetim
harikalar | etraftan, kitap
...
şey | konu, pürüzsüz
bozuldu | salıca, bihaber
aklım | gidiyorum, gibiyim
ay | tan

gitti | sürdü, majahahajajajajajajjjajajanznznznzjznz
ve | karne, görüntüler
şey | konu, pürüzsüz
ay | tanıdığım, duruyorsun
alnından | pasta, oysaki
aklım | gidiyorum, gibiyim
bir | konan, farkındayım
yerinde | meme, abim
tadından | seviyorummm, shshshsh
anlamlı | baktığında, sürünerek
isterim | veli, taşındık
 | soğukluğu, okula
namjoon | polise, yönetim
harikalar | etraftan, kitap
sesler | itin, has
perşembe | getirmiyo, ayak
...
gitti | sürdü, majahahajajajajajajjjajajanznznznzjznz
şey | konu, pürüzsüz
aklım | gidiyorum, gibiyim
ay | tanıdığım, duruyorsun
bir | konan, farkındayım
ve | karne, görüntüler
alnından | pasta, oysaki
bozuldu | salıca, sokmadınız
anlamlı | baktığında, sürünerek
harikalar | etraftan, kitap
dileriz | elettirik, koşulsuz
namjoon | polise, yönetim
tadından | seviyorummm, shshshsh
 | soğukluğu, okula
perşembe | getirmiyo, ayak
isterim | veli, vardım
...
gitti | sürdü, majahahajajajajajajjjajajanznznznzjznz
sıçradı | sıkkın, çıkamıyorum
aklım | gidiyorum, gibi

...

gitti | inanıyorum, netlik
sıçradı | görseli, paylaşamam
ve | içeriye, hissettiriyor
aklım | duvardan, dövme
ay | ney, sosyal
bozuldu | süpürgem, kopsaydı
şey | goygoycu, sürtük
yerinde | paramı, ablanın
olayım | manzarayı, doğuran
harikalar | yaratmışsın, ksjkssjnkdkjdk
dileriz | shshshshsksjsjsj, kahkaha
tadından | yenmiyor, teyzemizin
perşembe | akşamlarıca, yayınlanmış
namjoon | kahkahayla, yaşa
anlamlı | seninde, alexander
isterim | bassak, tanışmayı
...
yerinde | paramı, ablanın
sıçradı | görseli, paylaşamam
ay | ney, sosyal
ve | içeriye, hissettiriyor
bir | rybak, kalmamış
şey | goygoycu, sürtük
gitti | inanıyorum, netlik
bozuldu | süpürgem, kopsaydı
dileriz | shshshshsksjsjsj, kahkaha
perşembe | akşamlarıca, yayınlanmış
 | analizlerin, görürüm
harikalar | yaratmışsın, ksjkssjnkdkjdk
sesler | çıkardığım, hızlandırılması
anlamlı | seninde, alexander
namjoon | kahkahayla, yaşa
tadından | yenmiyor, teyzemizin
...
şey | goygoycu, sürtük
sıçradı | görseli, paylaşamam
aklım | duvarda

alnından | koluna, babayı»»»
ve | içeriye, yakınken
aklım | duvardan, dövme
yerinde | paramı, ablanın
gitti | inanıyorum, konuşması
ay | ney, sosyal
sıçradı | görseli, paylaşamam
bir | rybak, kalmamış
harikalar | yaratmışsın, ksjkssjnkdkjdk
 | görürüm, analizlerin
dileriz | shshshshsksjsjsj, kahkaha
tadından | yenmiyor, teyzemizin
perşembe | akşamlarıca, babasın
namjoon | kahkahayla, affet
isterim | bassak, tanışmayı
sesler | çıkardığım, demekti
...
alnından | koluna, babayı»»»
ay | ney, sosyal
gitti | inanıyorum, konuşması
şey | goygoycu, sürtük
sıçradı | görseli, paylaşamam
aklım | duvardan, dövme
bozuldu | süpürgem, kopsaydı
ve | içeriye, yakınken
 | görürüm, analizlerin
harikalar | yaratmışsın, ksjkssjnkdkjdk
dileriz | shshshshsksjsjsj, kahkaha
tadından | yenmiyor, teyzemizin
namjoon | kahkahayla, affet
isterim | bassak, tanışmayı
anlamlı | seninde, alexander
sesler | çıkardığım, demekti
...
alnından | koluna, babayı»»»
ay | ney, sosyal
gitti | inanıyorum, konuşması
şey | goygoyc

sıçradı | görseli, paylaşamam
yerinde | ablanın, paramı
bir | rybak, kalmamış
gitti | inanıyorum, konuşması
bozuldu | süpürgem, kopsaydı
şey | goygoycu, merhem
ve | içeriye, hissettiriyor
alnından | babayı»»», koluna
dileriz | shshshshsksjsjsj, kahkaha
tadından | icaz, yenmiyor
isterim | bassak, tanışmayı
namjoon | kahkahayla, yaşa
olayım | manzarayı, doğuran
harikalar | yaratmışsın, ksjkssjnkdkjdk
sesler | çıkardığım, demekti
perşembe | akşamlarıca, yayınlanmış
...
alnından | babayı»»», koluna
ve | içeriye, hissettiriyor
yerinde | ablanın, paramı
ay | ney, sosyal
aklım | duvardan, kendiliğinden
gitti | inanıyorum, konuşması
sıçradı | görseli, paylaşamam
şey | goygoycu, merhem
perşembe | akşamlarıca, yayınlanmış
olayım | manzarayı, doğuran
tadından | icaz, yenmiyor
anlamlı | alexander, seninde
sesler | çıkardığım, demekti
namjoon | kahkahayla, yaşa
 | analizlerin, görürüm
harikalar | yaratmışsın, ksjkssjnkdkjdk
...
bir | rybak, kalmamış
sıçradı | görseli, paylaşamam
ve | yakınken, içe

sıçradı | görseli, paylaşamam
aklım | duvardan, dövme
ay | ney, sosyal
ve | içeriye, hissettiriyor
bozuldu | süpürgem, kopsaydı
şey | goygoycu, merhem
gitti | inanıyorum, konuşması
bir | rybak, kalmamış
dileriz | shshshshsksjsjsj, kahkaha
olayım | manzarayı, doğuran
tadından | icaz, teyzemizin
anlamlı | alexander, seninde
namjoon | kahkahayla, affet
sesler | çıkardığım, demekti
 | analizlerin, görürüm
perşembe | akşamlarıca, babasın
...
aklım | duvardan, dövme
yerinde | ablanın, paramı
gitti | inanıyorum, netlik
alnından | koluna, babayı»»»
sıçradı | görseli, paylaşamam
bozuldu | süpürgem, kopsaydı
ve | içeriye, hissettiriyor
ay | ney, sosyal
tadından | icaz, teyzemizin
harikalar | yaratmışsın, ksjkssjnkdkjdk
dileriz | shshshshsksjsjsj, kahkaha
anlamlı | alexander, seninde
olayım | manzarayı, doğuran
sesler | çıkardığım, demekti
perşembe | akşamlarıca, babasın
namjoon | kahkahayla, affet
...
aklım | duvardan, dövme
bozuldu | süpürgem, kopsaydı
ve | hissettiriyor, içeriye
gitti | inanı

KeyboardInterrupt: 